In [ ]:
# https://www.kaggle.com/code/bunnyyy/t5-tuning-for-paraphrasing-questions/notebook

In [5]:
!pip install pytorch_lightning==0.8.1
!pip install transformers
!pip install SentencePiece 

# Po mojih izkušnjah je v Win10 pakete bolje inštalirati preko ukazne vrstice kot preko notebooka   

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.1/293.1 kB 23.9 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 107.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.1/200.1 kB 28.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 108.1 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 61.3 MB/s eta 0:00:00


In [1]:
import numpy as np 
import pandas as pd 

In [2]:
import argparse

In [3]:
import glob
import os
import json
import time
import logging
import random
import re
from itertools import chain
from string import punctuation


import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader

In [4]:
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)

device  cuda


In [5]:
from collections.abc import Mapping
import pytorch_lightning as pl
pl.__version__

'0.8.1'

In [6]:
from transformers import (
    AdamW,
    T5ForConditionalGeneration,
    T5Tokenizer,
    get_linear_schedule_with_warmup
)

In [7]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [8]:
!wget --no-check-certificate 'https://docs.google.com/uc?export=download&id=1x-LfRygUsTGPGAlYBiC47m6GNoq6C1H7' -O Deepl_trening.xlsx

--2023-04-15 14:38:58--  https://docs.google.com/uc?export=download&id=1x-LfRygUsTGPGAlYBiC47m6GNoq6C1H7
Resolving docs.google.com (docs.google.com)... 172.217.194.139, 172.217.194.138, 172.217.194.100, ...
Connecting to docs.google.com (docs.google.com)|172.217.194.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0o-7o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vsns7q6ptq1fsrq58goacnolj5a32845/1681569525000/15052681049997103435/*/1x-LfRygUsTGPGAlYBiC47m6GNoq6C1H7?e=download&uuid=45999cba-a038-408b-bbe7-57820b56a1b9 [following]
--2023-04-15 14:38:59--  https://doc-0o-7o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/vsns7q6ptq1fsrq58goacnolj5a32845/1681569525000/15052681049997103435/*/1x-LfRygUsTGPGAlYBiC47m6GNoq6C1H7?e=download&uuid=45999cba-a038-408b-bbe7-57820b56a1b9
Resolving doc-0o-7o-docs.googleusercontent.com (doc-0o-7o-docs.googleusercontent.com)... 142.251.12.132, 2404:6

In [9]:
skupna = pd.read_excel("Deepl_trening.xlsx")
originalniStavki = list(skupna.iloc[:,0])
parafrazeDeepl = list(skupna.iloc[:,2])

novaBaza = pd.DataFrame()

novaBaza["question"] = originalniStavki
novaBaza["target"] = parafrazeDeepl

train= novaBaza[:5500]
val= novaBaza[5501:6237]

In [10]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)


In [11]:
class T5FineTuner(pl.LightningModule):
    def __init__(self, hparams):
        super(T5FineTuner, self).__init__()
        self.hparams = hparams

        self.model = T5ForConditionalGeneration.from_pretrained(hparams.model_name_or_path)
        self.tokenizer = T5Tokenizer.from_pretrained(hparams.tokenizer_name_or_path)

    def is_logger(self):
        return True

    def forward(
            self, input_ids, attention_mask=None, decoder_input_ids=None, decoder_attention_mask=None, labels=None
    ):
        return self.model(
            input_ids,
            attention_mask=attention_mask,
            decoder_input_ids=decoder_input_ids,
            decoder_attention_mask=decoder_attention_mask,
            labels=labels,
        )

    def _step(self, batch):
        labels = batch["target_ids"]
        labels[labels[:, :] == self.tokenizer.pad_token_id] = -100

        outputs = self(
            input_ids=batch["source_ids"],
            attention_mask=batch["source_mask"],
            labels=labels,
            decoder_attention_mask=batch['target_mask']
        )

        loss = outputs[0]

        return loss

    def training_step(self, batch, batch_idx):
        loss = self._step(batch)

        tensorboard_logs = {"train_loss": loss}
        return {"loss": loss, "log": tensorboard_logs}

    def training_epoch_end(self, outputs):
        avg_train_loss = torch.stack([x["loss"] for x in outputs]).mean()
        tensorboard_logs = {"avg_train_loss": avg_train_loss}
        return {"avg_train_loss": avg_train_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def validation_step(self, batch, batch_idx):
        loss = self._step(batch)
        return {"val_loss": loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x["val_loss"] for x in outputs]).mean()
        tensorboard_logs = {"val_loss": avg_loss}
        return {"avg_val_loss": avg_loss, "log": tensorboard_logs, 'progress_bar': tensorboard_logs}

    def configure_optimizers(self):
        "Prepare optimizer and schedule (linear warmup and decay)"

        model = self.model
        no_decay = ["bias", "LayerNorm.weight"]
        optimizer_grouped_parameters = [
            {
                "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
                "weight_decay": self.hparams.weight_decay,
            },
            {
                "params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)],
                "weight_decay": 0.0,
            },
        ]
        optimizer = AdamW(optimizer_grouped_parameters, lr=self.hparams.learning_rate, eps=self.hparams.adam_epsilon)
        self.opt = optimizer
        return [optimizer]

    def optimizer_step(self, epoch, batch_idx, optimizer, optimizer_idx, second_order_closure=None):
        if self.trainer.use_tpu:
            xm.optimizer_step(optimizer)
        else:
            optimizer.step()
        optimizer.zero_grad()
        self.lr_scheduler.step()

    def get_tqdm_dict(self):
        tqdm_dict = {"loss": "{:.3f}".format(self.trainer.avg_loss), "lr": self.lr_scheduler.get_last_lr()[-1]}

        return tqdm_dict

    def train_dataloader(self):
        train_dataset = get_dataset(tokenizer=self.tokenizer, type_path="train", args=self.hparams)
        dataloader = DataLoader(train_dataset, batch_size=self.hparams.train_batch_size, drop_last=True, shuffle=True,
                                num_workers=4)
        t_total = (
                (len(dataloader.dataset) // (self.hparams.train_batch_size * max(1, self.hparams.n_gpu)))
                // self.hparams.gradient_accumulation_steps
                * float(self.hparams.num_train_epochs)
        )
        scheduler = get_linear_schedule_with_warmup(
            self.opt, num_warmup_steps=self.hparams.warmup_steps, num_training_steps=t_total
        )
        self.lr_scheduler = scheduler
        return dataloader

    def val_dataloader(self):
        val_dataset = get_dataset(tokenizer=self.tokenizer, type_path="valid", args=self.hparams)
        return DataLoader(val_dataset, batch_size=self.hparams.eval_batch_size, num_workers=4)

In [12]:
logger = logging.getLogger(__name__)

class LoggingCallback(pl.Callback):
        def on_validation_end(self, trainer, pl_module):
            logger.info("***** Validation results *****")
            if pl_module.is_logger():
                  metrics = trainer.callback_metrics
                  # Log results
                  for key in sorted(metrics):
                    if key not in ["log", "progress_bar"]:
                      logger.info("{} = {}\n".format(key, str(metrics[key])))

        def on_test_end(self, trainer, pl_module):
            logger.info("***** Test results *****")

            if pl_module.is_logger():
                metrics = trainer.callback_metrics

                  # Log and save results to file
                output_test_results_file = os.path.join(pl_module.hparams.output_dir, "test_results.txt")
                with open(output_test_results_file, "w") as writer:
                    for key in sorted(metrics):
                          if key not in ["log", "progress_bar"]:
                            logger.info("{} = {}\n".format(key, str(metrics[key])))
                            writer.write("{} = {}\n".format(key, str(metrics[key])))

In [13]:
args_dict = dict(
    data_dir="", # path for data files
    output_dir="", # path to save the checkpoints
    #model_name_or_path='ramsrigouthamg/t5_squad_v1',
    model_name_or_path='cjvt/t5-sl-small',
    tokenizer_name_or_path='cjvt/t5-sl-small',
    #max_seq_length=512,
    max_seq_length=50,
    learning_rate=3e-4,
    weight_decay=0.0,
    adam_epsilon=1e-8,
    warmup_steps=0,
    train_batch_size=32,
    eval_batch_size=32,
    num_train_epochs=100,
    gradient_accumulation_steps=32,
    n_gpu=1,
    early_stop_callback=False,
    fp_16=False,
    opt_level='O1', 
    max_grad_norm=1.0, 
    seed=42,
)

In [14]:
class GrammerDataset(Dataset):
    def __init__(self, tokenizer, data_dir, type_path, max_len=256):
        self.path = os.path.join(data_dir, type_path + '.csv')

        self.question = "question"
#         self.true_false = "answer"
        self.target_column = "target"
#         self.incorrect_max = "mix_in_correct"
        self.data = pd.read_csv(self.path)

        self.max_len = max_len
        self.tokenizer = tokenizer
        self.inputs = []
        self.targets = []

        self._build()

    def __len__(self):
        return len(self.inputs)

    def __getitem__(self, index):
        source_ids = self.inputs[index]["input_ids"].squeeze()
        target_ids = self.targets[index]["input_ids"].squeeze()

        src_mask = self.inputs[index]["attention_mask"].squeeze()  # might need to squeeze
        target_mask = self.targets[index]["attention_mask"].squeeze()  # might need to squeeze

        return {"source_ids": source_ids, "source_mask": src_mask, "target_ids": target_ids, "target_mask": target_mask}

    def _build(self):
        for idx in range(len(self.data)):
            target,question= self.data.loc[idx, self.target_column], self.data.loc[idx, self.question]
            #, self.data.loc[idx, self.incorrect_max]
#             true_false = str(true_false)
#             if true_false.lower() =="true":
#                 true_false ="yes"
#             else:
#                 true_false = "no"
#             input_ = ": "+ input_ + ' </s>'
            input_ = "ParaphraseQuestion: %s </s>" % (question)
            target = "ParaphrasedTarget: %s </s>" %(target)

            # tokenize inputs
            tokenized_inputs = self.tokenizer.batch_encode_plus(
                [input_], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
            )
            # tokenize targets
            tokenized_targets = self.tokenizer.batch_encode_plus(
                [target], max_length=self.max_len, pad_to_max_length=True, return_tensors="pt"
            )

            self.inputs.append(tokenized_inputs)
            self.targets.append(tokenized_targets)

In [15]:
train.to_csv('train.csv', index=False)
val.to_csv('valid.csv', index= False)

In [16]:
tokenizer = T5Tokenizer.from_pretrained('cjvt/t5-sl-small')

In [17]:
!mkdir result

In [18]:
args = argparse.Namespace(**args_dict)
print(args_dict)

{'data_dir': '', 'output_dir': '', 'model_name_or_path': 'cjvt/t5-sl-small', 'tokenizer_name_or_path': 'cjvt/t5-sl-small', 'max_seq_length': 50, 'learning_rate': 0.0003, 'weight_decay': 0.0, 'adam_epsilon': 1e-08, 'warmup_steps': 0, 'train_batch_size': 32, 'eval_batch_size': 32, 'num_train_epochs': 100, 'gradient_accumulation_steps': 32, 'n_gpu': 1, 'early_stop_callback': False, 'fp_16': False, 'opt_level': 'O1', 'max_grad_norm': 1.0, 'seed': 42}


In [19]:
checkpoint_callback = pl.callbacks.ModelCheckpoint(
    
    period =1,filepath=args.output_dir, prefix="checkpoint", monitor="val_loss", mode="min", save_top_k=1
)

train_params = dict(
    accumulate_grad_batches=args.gradient_accumulation_steps,
    gpus=args.n_gpu,
    max_epochs=args.num_train_epochs,
    early_stop_callback=False,
    precision= 16 if args.fp_16 else 32,
    amp_level=args.opt_level,
    gradient_clip_val=args.max_grad_norm,
    checkpoint_callback=checkpoint_callback,
    callbacks=[LoggingCallback()],
)

In [20]:
def get_dataset(tokenizer, type_path, args):
    return GrammerDataset(tokenizer=tokenizer, data_dir=args.data_dir, type_path=type_path,  max_len=args.max_seq_length)

In [21]:
print ("Initialize model")
model = T5FineTuner(args)

trainer = pl.Trainer(**train_params)

Initialize model


GPU available: True, used: True
INFO:lightning:GPU available: True, used: True
TPU available: False, using: 0 TPU cores
INFO:lightning:TPU available: False, using: 0 TPU cores
CUDA_VISIBLE_DEVICES: [0]
INFO:lightning:CUDA_VISIBLE_DEVICES: [0]


In [22]:
print (" Training model")
trainer.fit(model)

print ("training finished")

print ("Saving model")
model.model.save_pretrained("result")

print ("Saved model")

 Training model


/usr/local/lib/python3.9/dist-packages/transformers/optimization.py:391: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(

  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 76 M  
INFO:lightning:
  | Name  | Type                       | Params
-----------------------------------------------------
0 | model | T5ForConditionalGeneration | 76 M  
Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/u

Validation sanity check: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/transformers/tokenization_utils_base.py:2354: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/transformers/models/t5/tokenization_t5.py:226: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  warnings.warn(
/usr/local/lib/python3.9/dist-packages/torch/utils/data/dataloader.py:561: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoade

Training: 0it [00:00, ?it/s]

...

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

Validating: 0it [00:00, ?it/s]

training finished
Saving model
Saved model


In [25]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"
!zip shranjenModel.zip result/*.*

  adding: result/config.json (deflated 47%)
  adding: result/generation_config.json (deflated 29%)
  adding: result/pytorch_model.bin (deflated 10%)


In [ ]:
#-------------------------------------------
# Od tu naprej parafriziranje stavkov z narejenim modelom

In [26]:
stavkiZaPara = []
stavkiZaPara.append("Še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.")
stavkiZaPara.append("Bila so čisto navadna torkova večerna poročila na POP TV, torej oddaja 24ur, in je Darja Zgonc čisto mirno, kot da napoveduje, da vreme pač bo, izrekla naslednjo napoved prispevka.")
stavkiZaPara.append("Življenje samohranilke z dvema otrokoma se vrti nevarno blizu roba.")
stavkiZaPara.append("Janez Vogrinc je dekan Pedagoške fakultete v Ljubljani. ")
stavkiZaPara.append("Je tudi vodja delovne skupine za pripravo novega nacionalnega programa vzgoje in izobraževanja za obdobje 2023–2033, ki jo je oblikovalo resorno ministrstvo.")
stavkiZaPara.append("Glasbeni megahit, ki ga je še danes mogoče slišati na številnih plesiščih (ali pa vsaj po radijskih valovih), je opeval tradicionalno moške lastnosti in njihove adute.")
stavkiZaPara.append("Upravno sodišče je sredi letošnjega januarja izdalo odločbo, v kateri je ugodilo tožbi kmeta, ki mu je bilo razlaščeno 17.785 kvadratnih metrov kmetijskega zemljišča v korist Občine Hoče - Slivnica.")
stavkiZaPara.append("V okviru 14. Dneva za spremembe Slovenska filantropija po vsej Sloveniji organizira akcije, ki spodbujajo državljane k aktivnemu odzivu na potrebe svoje skupnosti.")
stavkiZaPara.append("Slovenijo je zajela protestniška inflacija.")
stavkiZaPara.append("Protestov je čedalje več, njihova vrednost pa je čedalje manjša.")

In [3]:
!rm -r model
!wget "https://drive.google.com/uc?id=1CGo_kjRT_i_BnWOd5NmZ3-0PaxASoMok&export=download&confirm=yes" -O model.zip
!unzip model.zip -d model

rm: cannot remove 'model': No such file or directory
--2023-04-15 14:31:37--  https://drive.google.com/uc?id=1G6q-v55Bbbn7ud6fjhmAOANfpx5QbZ6u&export=download&confirm=yes
Resolving drive.google.com (drive.google.com)... 142.251.12.138, 142.251.12.100, 142.251.12.101, ...
Connecting to drive.google.com (drive.google.com)|142.251.12.138|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://doc-0c-7o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cs39f9g1j5ltkbonuhtp59dj5hq8dnni/1681569075000/15052681049997103435/*/1G6q-v55Bbbn7ud6fjhmAOANfpx5QbZ6u?e=download&uuid=f7513d40-232b-45bb-9c7d-ba288f35a6e6 [following]
--2023-04-15 14:31:38--  https://doc-0c-7o-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/cs39f9g1j5ltkbonuhtp59dj5hq8dnni/1681569075000/15052681049997103435/*/1G6q-v55Bbbn7ud6fjhmAOANfpx5QbZ6u?e=download&uuid=f7513d40-232b-45bb-9c7d-ba288f35a6e6
Resolving doc-0c-7o-docs.googleusercontent.co

In [27]:
import torch
from transformers import T5ForConditionalGeneration,T5Tokenizer

def set_seed(seed):
  torch.manual_seed(seed)
  if torch.cuda.is_available():
    torch.cuda.manual_seed_all(seed)

set_seed(42)

#model = T5ForConditionalGeneration.from_pretrained('model')  # za download
model = T5ForConditionalGeneration.from_pretrained('result')  # za streniran
tokenizer = T5Tokenizer.from_pretrained('cjvt/t5-sl-small')

In [28]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print ("device ",device)
model = model.to(device)

device  cuda


In [29]:
for i in stavkiZaPara:
  text =  "paraphrase: " + i + " </s>"

  max_len = 256

  encoding = tokenizer.encode_plus(text,pad_to_max_length=True, return_tensors="pt")
  input_ids, attention_masks = encoding["input_ids"].to(device), encoding["attention_mask"].to(device)


  # set top_k = 50 and set top_p = 0.95 and num_return_sequences = 3
  beam_outputs = model.generate(
      input_ids=input_ids, attention_mask=attention_masks,
      do_sample=True,
      max_length=256,
      top_k=120,
      top_p=0.98,
      early_stopping=True,
      num_return_sequences=10
  )


  print ("\nOriginal Question ::")
  print (i)
  print ("\n")
  print ("Paraphrased Questions :: ")
  final_outputs =[]
  for beam_output in beam_outputs:
    sent = tokenizer.decode(beam_output, skip_special_tokens=True,clean_up_tokenization_spaces=True)
    if sent.lower() != i.lower() and sent not in final_outputs:
      final_outputs.append(sent)

  for i, final_output in enumerate(final_outputs):
    print("{}: {}".format(i, final_output))

  print("\n-----------------------\n")



Original Question ::
Še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.


Paraphrased Questions :: 
0: Paraphrased: Nobena vlada še nobeni vladi ni uspelo popraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.
1: Paraphrased: Nobena vlada še nobeni vladi ni uspelo odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici ni zavarovanje.
2: Paraphrased: Nobena vlada ni odpravila dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.
3: Paraphrased: Nobena vlada ni uspelo ukiniti dopolnilnega zdravstvenega zavarovanja, ki je dejansko zdravstveno zavarovanje.
4: ParaphrasedTarget: Nobena vlada ni uspela odpraviti dodatnega zdravstvenega zavarovanja, ki v resnici ni zavarovanje.
5: ParaphrasedTarget: Nobena vlada ni uspela odpraviti dopolnilnega zdravstvenega zavarovanja, ki v resnici sploh ni zavarovanje.
6: Paraphrased: Nobena vlada ni odpravila dopolnilnega zdravstvenega z